In [1]:
import face_recognition
import cv2
import numpy as np
import os
from skimage.io import imread

In [2]:
BASE_DIR = "C:\\Users\\91957\\Desktop\\i"
image_dir = os.path.join(BASE_DIR,"n")

In [3]:
video_capture = cv2.VideoCapture(0)

In [4]:
def get_encoded_faces():
    encoded = {}

    for dirpath, dnames, fnames in os.walk(image_dir): #OS.Walk() OS.walk() generate the file names in a directory tree by walking the tree either top-down or bottom-up. For each directory in the tree rooted at directory top (including top itself), it yields a 3-tuple (dirpath, dirnames, filenames).
        for f in fnames:
            if f.endswith(".jpg") or f.endswith(".png") or f.endswith(".jpeg") or f.endswith(".PNG") or f.endswith(".JPG") or f.endswith(".JPEG") :
                face = face_recognition.load_image_file("n/" + f)
                face = imread("n/" + f)
                encoding = face_recognition.face_encodings(face)[0]
                encoded[f.split(".")[0]] = encoding

    return encoded

In [5]:
def classify_face():
    
    faces = get_encoded_faces()
    
    known_face_encodings = list(faces.values())

    known_face_names = list(faces.keys())

    # Initialize some variable
    face_locations = []
    face_encodings = []
    face_names = []
    process_this_frame = True

    while True:
        ret, frame = video_capture.read()
        
        small_frame = cv2.resize(frame, (0,0), fx=0.25, fy=0.25) #to reduce the size
        #If you wish to use CV2, you need to use the resizefunction. For example, this will resize both axes by half: small = cv2.resize(image, (0,0), fx=0.5, fy=0.5)

        rgb_small_frame = small_frame[:,:,::-1]  #since the input image is in the form of bgr so we reverse the array so -1 to rgb 
        
        face_locations = face_recognition.face_locations(rgb_small_frame)
        face_encodings = face_recognition.face_encodings(rgb_small_frame, face_locations)

        for encodeFace,faceLoc in zip(face_encodings, face_locations):      #using zip to reduce the time complexity
            matches = face_recognition.compare_faces(known_face_encodings,encodeFace,0.5) #threshold 0.5 is for 50% of the image matching
            faceDis = face_recognition.face_distance(known_face_encodings, encodeFace) 
            matchIndex = np.argmin(faceDis) #Return indices of the minimum values along the given axis.
            #The numpy.argmin() method returns indices of the min element of the array in a particular axis. 
            name ="Unknown" 
            
            if matches[matchIndex]:
                name = known_face_names[matchIndex]
                y1,x2,y2,x1 = faceLoc
                y1,x2,y2,x1 = y1*4,x2*4,y2*4,x1*4   #since fx & fy =0.25 so, y1,x1*4  like that
                cv2.rectangle(frame, (x1,y1), (x2,y2), (0,255,0),2)
                cv2.rectangle(frame,(x1,y2-35), (x2,y2), (0,255,0), cv2.FILLED) #two frames for two one for bigger box another where name is dispayed that box with thickness 2 
                cv2.putText(frame, name, (x1-6,y2-6),cv2.FONT_HERSHEY_COMPLEX,0.5,(255,255,255),2)
            if not matches[matchIndex]:
                y1,x2,y2,x1 = faceLoc
                y1,x2,y2,x1 = y1*4,x2*4,y2*4,x1*4
                cv2.rectangle(frame, (x1,y1), (x2,y2), (0,255,0),2)
                cv2.rectangle(frame,(x1,y2-35), (x2,y2), (0,255,0), cv2.FILLED)
                cv2.putText(frame, name, (x1-6,y2-6),cv2.FONT_HERSHEY_COMPLEX,0.5,(255,255,255),2)


        # Display Result
        cv2.imshow("Video", frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    video_capture.release()
    cv2.destroyAllWindows()

In [ ]:
print(classify_face())

In [ ]:
for dirpath, dnames, fnames in os.walk(image_dir):
        print(fnames)
        